# Генерация каналов и сигналов

- Перевод волатильности в уровни каналов: ±1σ, ±2σ
- Логика входа/выхода по сигналам в зависимости от тренда
- Создание DataFrame для анализа сигналов

Подготовительная часть для последующего бэктеста.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("✅ Библиотеки загружены")


In [ ]:
def create_volatility_channels(price, volatility_forecast, n_sigmas=[1, 2]):
    """
    Создание торговых каналов на основе прогноза волатильности
    """
    channels = pd.DataFrame(index=price.index)
    channels['price'] = price
    channels['vol_forecast'] = volatility_forecast
    
    for sigma in n_sigmas:
        channels[f'upper_{sigma}sigma'] = price + (volatility_forecast * sigma)
        channels[f'lower_{sigma}sigma'] = price - (volatility_forecast * sigma)
    
    return channels


def generate_signals(channels, trend_signal=None):
    """
    Генерация торговых сигналов на основе каналов
    1 = Long, -1 = Short, 0 = No position
    """
    signals = pd.DataFrame(index=channels.index)
    signals['price'] = channels['price']
    
    # Пробой верхней границы канала (1σ) - продажа
    signals['sell_signal'] = (channels['price'] > channels['upper_1sigma']).astype(int)
    
    # Пробой нижней границы канала (1σ) - покупка
    signals['buy_signal'] = (channels['price'] < channels['lower_1sigma']).astype(int)
    
    # Комбинированный сигнал с учетом тренда
    if trend_signal is not None:
        # В uptrend только покупки, в downtrend только продажи
        signals['buy_signal'] = signals['buy_signal'] * (trend_signal >= 0).astype(int)
        signals['sell_signal'] = signals['sell_signal'] * (trend_signal <= 0).astype(int)
    
    # Финальный сигнал позиции
    signals['position'] = signals['buy_signal'] - signals['sell_signal']
    
    return signals

print("✅ Функции каналов и сигналов загружены")


## Загрузка данных и создание каналов


In [ ]:
# Загрузка прогнозов ансамбля
MODEL_DIR = Path('data') / 'models'
ticker = 'SBER'

ensemble_df = pd.read_parquet(MODEL_DIR / f"{ticker}_ensemble_forecasts.parquet")
print(f"✅ Данные загружены: {len(ensemble_df)} записей")

# Создаем каналы на основе прогноза ансамбля
channels = create_volatility_channels(
    ensemble_df['actual'],  # Используем актуальную цену
    ensemble_df['ensemble_forecast'],  # Прогноз волатильности
    n_sigmas=[1, 2]
)

print("\nПример каналов:")
print(channels.tail())


## Генерация сигналов


In [ ]:
# Генерация сигналов (без учета тренда для простоты)
signals = generate_signals(channels, trend_signal=None)

print(f"\n✅ Сигналы сгенерированы")
print(f"Количество Buy сигналов: {signals['buy_signal'].sum()}")
print(f"Количество Sell сигналов: {signals['sell_signal'].sum()}")
print(f"\nПример сигналов:")
print(signals.tail())


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'backtest'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Объединяем каналы и сигналы
backtest_data = pd.concat([channels, signals[['buy_signal', 'sell_signal', 'position']]], axis=1)

output_path = OUTPUT_DIR / f"{ticker}_signals_channels.parquet"
backtest_data.to_parquet(output_path, index=False)

print(f"✅ Сохранено: {output_path}")
